In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/NLP-2-main.zip"

extract_path = "/content/NLP-2-extracted"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to {extract_path}")

Files extracted to /content/NLP-2-extracted


In [ ]:

print("Contents of the extracted folder:")
for item in os.listdir(extract_path):
    print(item)

Contents of the extracted folder:
NLP-2-main


In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
import os
import openai
# from getpass import getpass

api_key = 'sk-9zdEnnpoNRxNaCvWJS3'


client = openai.OpenAI(
    api_key=api_key,
    base_url="https://cmu.litellm.ai"
)


def get_completion(prompt, model="gpt-4o"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

while True:
    user_input = input("\nEnter your question (or 'quit' to exit): ")
    if user_input.lower() == 'quit':
        break
    response = get_completion(user_input)
    print("\nResponse:", response)

print("Thank you for using the LTI LiteLLM API Proxy!")

Hello! How can I assist you today?

Enter your question (or 'quit' to exit): Hi

Response: Hello! How can I assist you today?

Enter your question (or 'quit' to exit): quit
Thank you for using the LTI LiteLLM API Proxy!


In [ ]:
import os
import openai
from nltk.tokenize import sent_tokenize
import nltk

api_key = 'sk-9zdEnnpoNRxNaCvWJS36Vg'
client = openai.OpenAI(
    api_key=api_key,
    base_url="https://cmu.litellm.ai"
)

def get_completion(prompt, model="gpt-4o"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

def chunk_text(text, chunk_size=10000):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= chunk_size:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

def generate_qa_pair(chunk):
    prompt = f"""Based on the following text, generate a single-line question and its answer. The question should be specific and answerable from the given text.

Example:
Text: The Eiffel Tower, located in Paris, France, was completed in 1889. It stands at a height of 324 meters and was the tallest man-made structure in the world for 41 years until the Chrysler Building in New York City was built in 1930.
Q: In what year was the Eiffel Tower completed?
A: The Eiffel Tower was completed in 1889.

Now, generate a question and answer for the following text:
{chunk}"""
    response = get_completion(prompt)
    qa_pair = response.split('\n', 1)
    if len(qa_pair) == 2:
        return qa_pair[0].strip(), qa_pair[1].strip()
    else:
        return None, None

def process_documents(folder_path, output_file):
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for filename in os.listdir(folder_path):
            if filename.endswith('.txt'):
                file_path = os.path.join(folder_path, filename)
                print(f"Processing: {filename}")
                with open(file_path, 'r', encoding='utf-8') as infile:
                    content = infile.read()
                chunks = chunk_text(content)
                outfile.write(f"File: {filename}\n\n")
                for chunk in chunks:
                    question, answer = generate_qa_pair(chunk)
                    if question and answer:
                        outfile.write(f"Q: {question}\n")
                        outfile.write(f"A: {answer}\n\n")
                outfile.write("="*50 + "\n\n")
                print(f"Completed: {filename}")

folder_path = '/content/NLP-2-extracted/NLP-2-main/raw_data/Sports'
output_file = '/content/test_set_sports.txt'

print('Process started')
process_documents(folder_path, output_file)
print(f"Q&A pairs have been generated and saved to {output_file}")
print('Process completed')

Process started
Processing: penguins.txt
Completed: penguins.txt
Processing: steelers.txt
Completed: steelers.txt
Processing: visitpittsburgh_copy.txt
Completed: visitpittsburgh_copy.txt
Processing: pirates.txt
Completed: pirates.txt
Q&A pairs have been generated and saved to /content/test_set_sports.txt
Process completed
